In [2]:
import datetime
import requests
import json
import pandas as pd

# Example coordinates
longitude = -122.4194  # San Francisco
latitude = 37.7749

# Calculate end date (previous year to ensure complete data)
endtime = str(int(datetime.date.today().strftime("%Y")) - 1) + '1231'

# Construct API request URL
base_url = (
    r"https://power.larc.nasa.gov/api/temporal/daily/point?"
    r"parameters=T2M_MAX,PRECTOTCORR,T2M_MIN,RH2M,ALLSKY_SFC_SW_DWN&community=RE&"
    r"longitude={longitude}&latitude={latitude}&"
    r"start=19810101&end=" + endtime + "&format=JSON"
)

api_request_url = base_url.format(
    longitude=longitude, 
    latitude=latitude
)

# Prepare proxy settings for requests (set to empty string if not needed)
proxy = ""  # e.g., "http://yourproxy:port"
proxies = {}
if proxy:
    proxies = {
        'http': proxy,
        'https': proxy
    }
    print(f"Using proxy: {proxy}")

# Make API request with timeout and proxy support
response = None
try:
    if proxies:
        print("Attempting request with proxy...")
        response = requests.get(
            url=api_request_url, 
            verify=True, 
            timeout=100,
            proxies=proxies
        )
    else:
        # No proxy configured, make direct request
        response = requests.get(
            url=api_request_url, 
            verify=True, 
            timeout=100
        )
except Exception as e:
    if proxies:
        print(f"Request with proxy failed: {e}")
        print("Retrying without proxy...")
        try:
            response = requests.get(
                url=api_request_url, 
                verify=True, 
                timeout=100
            )
            print("Request successful without proxy")
        except Exception as e2:
            print(f"Request failed even without proxy: {e2}")
            raise e2
    else:
        print(f"Direct request failed: {e}")
        raise e

content = json.loads(response.content.decode('utf-8'))

# Convert to DataFrame and rename columns
df = pd.DataFrame.from_dict(content['properties']['parameter'])
df = df.reset_index().rename(columns={
    'index': 'Date', 
    'PRECTOTCORR': 'Precipitation', 
    'T2M_MIN': 'Min Temperature', 
    'T2M_MAX': 'Max Temperature'
})

# Convert date column to datetime
df.Date = pd.to_datetime(df.Date)

print('Climate data request completed successfully')
print(df.head())


Climate data request completed successfully
        Date  Max Temperature  Precipitation  Min Temperature   RH2M  \
0 1981-01-01            17.16           0.10             5.31  69.21   
1 1981-01-02            12.53           3.68             4.43  76.80   
2 1981-01-03            14.30           1.37             9.01  91.73   
3 1981-01-04            16.79           0.05             7.51  86.71   
4 1981-01-05            17.01           0.00             5.15  78.53   

   ALLSKY_SFC_SW_DWN  
0             -999.0  
1             -999.0  
2             -999.0  
3             -999.0  
4             -999.0  


In [6]:
test = [1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

In [7]:
test[19]



2000

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16071 entries, 0 to 16070
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               16071 non-null  datetime64[ns]
 1   Max Temperature    16071 non-null  float64       
 2   Precipitation      16071 non-null  float64       
 3   Min Temperature    16071 non-null  float64       
 4   RH2M               16071 non-null  float64       
 5   ALLSKY_SFC_SW_DWN  14976 non-null  float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 753.5 KB
